# 空氣污染網站爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

* 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

## 1. 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [25]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")


In [7]:
selectSite = Select(browser.find_element_by_id('ctl05_ddlSite'))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id('ctl05_ddlYear'))
selectYear.select_by_value('2018')
browser.find_element_by_id('ctl05_btnQuery').click()

In [31]:
html_source = browser.page_source
soup = BeautifulSoup(html_source,'html5lib')
trs = soup.find('table',{'class':'TABLE_G'}).find_all('tr')
rowspan = 0
count = 0
SO2 = {
    '監測日期':[],
    '監測值':[]
}
for tr in trs:
    if 'SO2' in tr.text:
        rowspan = int(tr.td['rowspan'])
#         print(rowspan)
    if rowspan != 0:
        tds = tr.find_all('td',{'class':None})
        if int(tds[0].text[5:7]) >=1 and int(tds[0].text[5:7]) <=8:
            SO2['監測日期'].append(tds[0].text)
            SO2['監測值'].append(tds[1].text)
        count = count + 1
        if count == rowspan:
            break


In [32]:
pd.DataFrame(SO2)

,監測日期,監測值
0,2018/01,1.80
1,2018/02,1.90
2,2018/03,2.20
3,2018/04,2.30
4,2018/05,3.10
5,2018/06,2.70
6,2018/07,2.20
7,2018/08,2.40


## 2. 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [80]:
html_source = browser.page_source
soup = BeautifulSoup(html_source,'html5lib')
trs = soup.find('table',{'class':'TABLE_G'}).find_all('tr')
rowspan = 0
COcount = 0
SO2count = 0
count = 0
SO2 = {
    '監測日期':[],
    'SO2-監測值':[],
    'CO-監測值':[]
}
test = ['SO2','CO']
word = ''
for tr in trs:
    if len(tr.find_all('td',{'class':'no-alt'})) != 0:
#         print(tr.td.text)
#         print(tr)
        word = tr.td.text
    if word in test:
#         print(tr)
        rowspan = int(tr.td['rowspan'])
        word = ''
    if rowspan !=0:
        if SO2count != rowspan:
            tds = tr.find_all('td',{'class':None})
            if int(tds[0].text[5:7]) >=1 and int(tds[0].text[5:7]) <=8:
                SO2['監測日期'].append(tds[0].text)
                SO2['SO2-監測值'].append(tds[1].text)
#                 print(SO2)
            SO2count = SO2count + 1
        elif COcount != rowspan:
            tds = tr.find_all('td',{'class':None})
            if int(tds[0].text[5:7]) >=1 and int(tds[0].text[5:7]) <=8:
                SO2['CO-監測值'].append(tds[1].text)
            COcount = COcount + 1
        else:
            break


{'監測日期': ['2018/01', '2018/02', '2018/03', '2018/04', '2018/05', '2018/06', '2018/07', '2018/08'], 'SO2-監測值': ['1.80', '1.90', '2.20', '2.30', '3.10', '2.70', '2.20', '2.40'], 'CO-監測值': ['0.34', '0.44', '0.40', '0.38', '0.34', '0.29', '0.21', '0.30']}


In [81]:
pd.DataFrame(SO2)

,監測日期,SO2-監測值,CO-監測值
0,2018/01,1.80,0.34
1,2018/02,1.90,0.44
2,2018/03,2.20,0.40
3,2018/04,2.30,0.38
4,2018/05,3.10,0.34
5,2018/06,2.70,0.29
6,2018/07,2.20,0.21
7,2018/08,2.40,0.30
